Task4_0603 train.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요.

In [158]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

# 업로드된 파일을 데이터프레임으로 읽기
file_path = list(uploaded.keys())[0]
df = pd.read_csv(file_path)

Saving train.csv to train (9).csv


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [113]:
df_numeric = df.select_dtypes(include='number')
df_numeric.corr()['Survived']

PassengerId   -0.005007
Survived       1.000000
Pclass        -0.338481
Age           -0.077221
SibSp         -0.035322
Parch          0.081629
Fare           0.257307
Name: Survived, dtype: float64

In [163]:
# 필요없는 열 제거
# df.drop(columns = ['PassengerId','Name','Ticket','Cabin','Pclass','Embarked'], inplace=True)
df.drop(columns = ['PassengerId','Name','Ticket','Cabin','Fare'], inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [164]:
# 결측치 확인
print(df.isnull().sum())

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64


In [167]:
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True)

In [168]:
# 결측치 제거
print(df.isnull().sum())
df.Age.fillna(df.Age.mean(), inplace=True)
print(df.isnull().sum())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64
Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64


In [169]:
# 파생변수 생성
df['family_size'] = df['SibSp']+df['Parch']+1
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,family_size
0,0,3,male,22.0,1,0,S,2
1,1,1,female,38.0,1,0,C,2
2,1,3,female,26.0,0,0,S,1
3,1,1,female,35.0,1,0,S,2
4,0,3,male,35.0,0,0,S,1


In [170]:
# 필요없는 열 제거
df.drop(columns=['SibSp','Parch'],inplace=True)
df.head()

,Survived,Pclass,Sex,Age,Embarked,family_size
0,0,3,male,22.0,S,2
1,1,1,female,38.0,C,2
2,1,3,female,26.0,S,1
3,1,1,female,35.0,S,2
4,0,3,male,35.0,S,1


In [171]:
# age_group을 6개 범주로 확대
bins = [0,5,12,18,35,60,100]
labels = ['Infant','Child','Teenager','Yound Adult','Adult','Senior']
df['age_group'] = pd.cut(df['Age'], bins=bins, labels = labels)
df.drop(columns=['Age'], inplace = True)
df.head()

,Survived,Pclass,Sex,Embarked,family_size,age_group
0,0,3,male,S,2,Yound Adult
1,1,1,female,C,2,Adult
2,1,3,female,S,1,Yound Adult
3,1,1,female,S,2,Yound Adult
4,0,3,male,S,1,Yound Adult


In [172]:
category_column = ['age_group','Sex','family_size','Pclass','Embarked']
for column in category_column:
    df= pd.get_dummies(df,columns=[column])


In [173]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# 타겟 변수와 피처 변수 설정
X = df.drop('Survived', axis=1)
y = df['Survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


(712, 23) (179, 23) (712,) (179,)
Accuracy: 0.8268156424581006
Confusion Matrix:
 [[96  9]
 [22 52]]
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.91      0.86       105
           1       0.85      0.70      0.77        74

    accuracy                           0.83       179
   macro avg       0.83      0.81      0.82       179
weighted avg       0.83      0.83      0.82       179

